<a href="https://colab.research.google.com/github/Xin-overclocked/pns/blob/main/huisan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Section 1: Define Your Research Question and Dataset**


**Research Questions**


1.   How does the age of athletes influence their likelihood of winning a medal in the Olympics across various sports?
2.   To what extent does an athlete's weight impact their chances of securing a medal in the Olympics, considering different sports categories?

In [10]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/athlete_events.csv')

Briefly describe your dataset, including its source, the main variables, sample size, and any limitations.


**Source:**

The dataset is sourced from Kaggle and it contains information on Olympic athletes.  The specific file used is 'athlete_events.csv'.

**Main Variables:**

The dataset includes key variables such as athlete information (ID, name, sex, age, height, weight), game details (year, season, city), sport and event information, and medal outcomes (gold, silver, bronze).

**Limitations:**

* Missing Data:  The presence of missing values (NaNs) in columns like 'Age', 'Height', and 'Weight' may introduce bias or necessitate imputation.

* Data Accuracy:  Data accuracy from historical sources may vary.

* Representativeness: The dataset may not accurately represent all athletes who have ever participated in the Olympics throughout history.

* Scope: Limited to recorded data points in the dataset and may not cover all aspects of an athlete's performance or contextual factors.


In [13]:
print(f"Sample Size: {df.shape[0]}")

Sample Size: 271116


# **Section 2: Summarize Key Data with Descriptive Statistics**

# New section